<a href="https://colab.research.google.com/github/duttaprat/BMI_503/blob/main/class_1/notebook2_transcriptomics_packages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Class 1 - Notebook 2: Python Packages for Transcriptomics

**Course**: BMI 503 - Introduction to Computer Science for Biomedical Informatics  
**Instructor**: Pratik Dutta  
**Term**: Fall 2025  
**Institution**: Stony Brook University

---

## Learning Objectives
By the end of this notebook, you will be able to:
1. Understand different types of transcriptomics data
2. Use Python packages to load and analyze RNA-seq data
3. Work with count matrices and expression data
4. Perform basic single-cell RNA-seq analysis
5. Visualize transcriptomics data

## Introduction: What is Transcriptomics?

**Transcriptomics** = Study of all RNA transcripts in a cell or tissue

### Central Dogma of Molecular Biology

```
DNA → (Transcription) → RNA → (Translation) → Protein
     🧬                  📊                    🧪
   Genomics          TRANSCRIPTOMICS        Proteomics
```

**Why study the transcriptome?**
- Shows which genes are ACTIVE (not just present)
- Reveals cell state and function
- Changes in disease, development, treatment
- More dynamic than genome

## Types of Transcriptomics

### 1. **Bulk RNA-seq**
- Average gene expression across ALL cells in sample
- Lost: Cell-to-cell variation
- Use: Compare tissues, conditions, treatments

### 2. **Single-Cell RNA-seq (scRNA-seq)**
- Gene expression in INDIVIDUAL cells
- Reveals: Cell types, states, heterogeneity
- Use: Cell atlas, rare populations, development

### 3. **Spatial Transcriptomics**
- Gene expression + WHERE in tissue
- Preserves: Tissue architecture
- Use: Tissue organization, cell interactions

## Python Packages for Transcriptomics

| Package | Purpose | Data Type |
|---------|---------|----------|
| `pandas` | Data manipulation | Count matrices |
| `numpy` | Numerical operations | Arrays, matrices |
| `matplotlib/seaborn` | Visualization | All types |
| `scipy` | Statistical analysis | Expression data |
| `scanpy` ⭐ | scRNA-seq analysis | Single-cell |
| `anndata` | Annotated data structures | Single-cell |
| `gseapy` | Pathway enrichment | Gene lists |

## Setup: Install Packages

In [1]:
!pip install pandas numpy matplotlib seaborn scipy scanpy anndata gseapy -q
print("✅ All packages installed successfully!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.7/170.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.2/58.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.0/80.0 kB 2.0 MB/s eta 0:00:00
✅ All packages installed successfully!


In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import scanpy as sc
import anndata as ad
import gseapy as gp

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
sc.settings.verbosity = 1

print("📦 All libraries loaded!")
print(f"\npandas: {pd.__version__}")
print(f"numpy: {np.__version__}")
print(f"scanpy: {sc.__version__}")

📦 All libraries loaded!

pandas: 2.2.2
numpy: 2.0.2
scanpy: 1.11.5


---
# Part 1: Bulk RNA-seq Data Analysis

## 1.1: Creating Synthetic RNA-seq Data

In [3]:
np.random.seed(42)
n_genes = 1000
gene_names = [f'Gene_{i:04d}' for i in range(n_genes)]
control_samples = [f'Control_{i}' for i in range(1, 4)]
treated_samples = [f'Treated_{i}' for i in range(1, 4)]
all_samples = control_samples + treated_samples

base_expression = np.random.negative_binomial(10, 0.1, (n_genes, 6))
base_expression[0:50, 3:6] *= 5
base_expression[50:100, 3:6] //= 5

counts = pd.DataFrame(base_expression, index=gene_names, columns=all_samples)
print("📊 RNA-seq Count Matrix:")
print(counts.head(10))
print(f"\nShape: {counts.shape[0]} genes × {counts.shape[1]} samples")

📊 RNA-seq Count Matrix:
           Control_1  Control_2  Control_3  Treated_1  Treated_2  Treated_3
Gene_0000        104         86         65        505        290        435
Gene_0001         63         84         71        705        535        160
Gene_0002         77         73         87        410        340        370
Gene_0003        111        140         84        660        440        455
Gene_0004        116         91         87        250        365        315
Gene_0005         44        101         79        260        670        245
Gene_0006         82         79         78        435        260        530
Gene_0007         82         79        140        480        595        460
Gene_0008         88         99        102        315        305        325
Gene_0009         65         97         66        355        555        545

Shape: 1000 genes × 6 samples


## 1.2: Basic Statistics

In [ ]:
print("📊 SUMMARY STATISTICS:")
print(counts.describe())
library_sizes = counts.sum(axis=0)
print("\n📚 Library Sizes:")
print(library_sizes)
genes_detected = (counts > 0).sum(axis=0)
print("\n🔬 Genes Detected:")
print(genes_detected)

## 1.3: Quality Control Visualizations

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
axes[0].bar(range(len(library_sizes)), library_sizes, color=['blue']*3+['red']*3)
axes[0].set_xticks(range(len(all_samples)))
axes[0].set_xticklabels(all_samples, rotation=45, ha='right')
axes[0].set_ylabel('Total Counts', fontweight='bold')
axes[0].set_title('Library Sizes')

for i, sample in enumerate(all_samples):
    color = 'blue' if i < 3 else 'red'
    axes[1].hist(np.log10(counts[sample] + 1), bins=50, alpha=0.3, color=color)
axes[1].set_xlabel('Log10(Count + 1)')
axes[1].set_title('Count Distribution')

sns.heatmap(counts.corr(), annot=True, fmt='.2f', cmap='coolwarm', ax=axes[2])
axes[2].set_title('Sample Correlation')
plt.tight_layout()
plt.show()

## 1.4: Normalization

In [ ]:
def cpm_normalize(counts_df):
    total_counts = counts_df.sum(axis=0)
    return (counts_df / total_counts) * 1e6

cpm_data = cpm_normalize(counts)
log_cpm = np.log2(cpm_data + 1)
print("📊 CPM Normalized:")
print(cpm_data.head())
print("\n📊 Log2 Transformed:")
print(log_cpm.head())

## 1.5: Differential Expression

In [ ]:
control_mean = log_cpm[control_samples].mean(axis=1)
treated_mean = log_cpm[treated_samples].mean(axis=1)
log_fold_change = treated_mean - control_mean

p_values = []
for gene in counts.index:
    _, p_val = stats.ttest_ind(log_cpm.loc[gene, control_samples], log_cpm.loc[gene, treated_samples])
    p_values.append(p_val)

de_results = pd.DataFrame({
    'gene': counts.index,
    'log2FC': log_fold_change.values,
    'pvalue': p_values,
    'control_mean': control_mean.values,
    'treated_mean': treated_mean.values
})
de_results['padj'] = (de_results['pvalue'] * len(de_results)).clip(upper=1.0)
de_results['significant'] = (de_results['padj'] < 0.05) & (abs(de_results['log2FC']) > 1)
print(de_results.head())
print(f"\n🔍 Significant genes: {de_results['significant'].sum()}")

## 1.6: Volcano Plot

In [ ]:
plt.figure(figsize=(10, 8))
ns = de_results[~de_results['significant']]
plt.scatter(ns['log2FC'], -np.log10(ns['pvalue']), c='gray', alpha=0.3, s=10, label='NS')
sig = de_results[de_results['significant']]
plt.scatter(sig['log2FC'], -np.log10(sig['pvalue']), c='red', alpha=0.6, s=20, label='Sig')
plt.axhline(-np.log10(0.05), color='blue', linestyle='--', alpha=0.5)
plt.axvline(-1, color='green', linestyle='--', alpha=0.5)
plt.axvline(1, color='green', linestyle='--', alpha=0.5)
plt.xlabel('Log2 Fold Change')
plt.ylabel('-Log10(p-value)')
plt.title('Volcano Plot')
plt.legend()
plt.show()
print(f"🔺 Up: {(sig['log2FC'] > 1).sum()}")
print(f"🔻 Down: {(sig['log2FC'] < -1).sum()}")

---
# Part 2: Single-Cell RNA-seq

## 2.1: Load scRNA-seq Data

In [ ]:
adata = sc.datasets.pbmc3k()
print(f"📦 Loaded!")
print(f"Cells: {adata.n_obs}")
print(f"Genes: {adata.n_vars}")

## 2.2: Quality Control

In [ ]:
sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
axes[0].hist(adata.obs['total_counts'], bins=50, color='skyblue')
axes[0].set_title('UMI Counts')
axes[1].hist(adata.obs['n_genes_by_counts'], bins=50, color='coral')
axes[1].set_title('Genes Detected')
axes[2].scatter(adata.obs['total_counts'], adata.obs['n_genes_by_counts'], s=5, alpha=0.5)
axes[2].set_title('QC Scatter')
plt.tight_layout()
plt.show()

## 2.3: Filtering

In [ ]:
print(f"Before: {adata.n_obs} cells")
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
print(f"After: {adata.n_obs} cells, {adata.n_vars} genes")

## 2.4: Normalization

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
print("✅ Normalized and log-transformed")

## 2.5: Highly Variable Genes

In [ ]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000)
sc.pl.highly_variable_genes(adata)
print(f"🧬 HVGs: {adata.var['highly_variable'].sum()}")

## 2.6: PCA

In [ ]:
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, n_pcs=50)
print("✅ PCA done")

## 2.7: UMAP

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
fig, axes = plt.subplots(1, 2, figsize=(16, 7))
sc.pl.umap(adata, color='total_counts', ax=axes[0], show=False)
sc.pl.umap(adata, color='n_genes_by_counts', ax=axes[1], show=False)
plt.show()
print("✅ UMAP done")

## 2.8: Clustering

In [ ]:
sc.tl.leiden(adata, resolution=0.5)
sc.pl.umap(adata, color='leiden', legend_loc='on data')
print(f"🎯 Clusters: {adata.obs['leiden'].nunique()}")

## 2.9: Marker Genes

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=10, sharey=False)
print("🔬 Marker genes found")

## 2.10: Visualize Markers

In [ ]:
marker_genes = ['CD79A', 'MS4A1', 'CD8A', 'CD4', 'IL7R', 'CD14', 'LYZ']
available = [g for g in marker_genes if g in adata.var_names]
if available:
    sc.pl.dotplot(adata, available, groupby='leiden')
    sc.pl.umap(adata, color=available[:4], ncols=2)
else:
    print("⚠️ Markers not in dataset")

---
# Part 3: Pathway Enrichment

## 3.1: Prepare Gene List

In [ ]:
upregulated = de_results[(de_results['significant']) & (de_results['log2FC'] > 1)]['gene'].tolist()
print(f"🧬 Upregulated: {len(upregulated)}")
print(upregulated[:10])

## 3.2: Run Enrichment

In [ ]:
try:
    enr = gp.enrichr(gene_list=upregulated, gene_sets='GO_Biological_Process_2021', organism='Human', outdir=None)
    results = enr.results.sort_values('Adjusted P-value')
    print("✅ Enrichment done")
    print(results[['Term', 'Adjusted P-value']].head(10))

    top = results.head(15)
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(top)), -np.log10(top['Adjusted P-value']), color='steelblue')
    plt.yticks(range(len(top)), top['Term'], fontsize=10)
    plt.xlabel('-Log10(Adj P-value)')
    plt.title('Top GO Terms')
    plt.show()
except Exception as e:
    print(f"⚠️ Requires internet: {e}")

---
# Summary

## What We Learned

### Part 1: Bulk RNA-seq
- pandas for count matrices
- CPM normalization
- Differential expression
- Volcano plots

### Part 2: Single-Cell
- scanpy workflow
- AnnData objects
- QC and filtering
- PCA, UMAP, clustering
- Marker genes

### Part 3: Pathways
- gseapy enrichment
- Biological interpretation

## Key Packages

| Package | Purpose |
|---------|---------||
| pandas | Data manipulation |
| numpy | Numerical ops |
| scipy | Statistics |
| scanpy | scRNA-seq |
| gseapy | Enrichment |

## Exercises

1. Change FC threshold to 2
2. Try resolution=0.3 in clustering
3. Find markers for cluster 1
4. Create MA plot
5. Try KEGG database

## Resources

- Scanpy: https://scanpy.readthedocs.io/
- Pandas: https://pandas.pydata.org/
- GSEApy: https://gseapy.readthedocs.io/

---

**🎓 Class 1 Complete!**

You now know Python packages for:
1. ✅ Genomics
2. ✅ Transcriptomics  
3. ✅ Imagomics